In [6]:
import os
import json
import pandas as pd
from langchain.globals import set_debug
set_debug(False)


# 設定 APIKEY
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [8]:
with open('cxi_filter_traffic_0506.json', 'r') as f:
    data = json.load(f)
data = pd.DataFrame(data)
data = data[['jid', 'jfull_compress']]

In [9]:
n_content = 10
data_folder = 'data'
if data_folder not in os.listdir():
    os.mkdir('data')
    
for idx, item in data.iloc[:n_content].iterrows():
    file_name = item['jid'] + '.txt'
    content = item['jfull_compress']
    with open(f'./{data_folder}/{file_name}', 'w') as f:
        f.write(content)

In [10]:
# linux install run $ curl -fsSL https://ollama.com/install.sh | sh
# 要使用模型需要先下載 run $ ollama pull llama2
ollama_host = "localhost"
ollama_port = 11434
# ollama_model = "llama2"
# ollama_model = "qwen:7b"
ollama_model = "ryan4559/llama3-taide-lx-8b-chat-alpha1-4bit:latest"

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model=ollama_model, 
                 temperature=0)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model_id = "intfloat/multilingual-e5-large"
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

In [12]:
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd
n_data = 10
loader = DataFrameLoader(data.iloc[:n_data],
                         page_content_column="jfull_compress")
documents = loader.load()

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False)

splitted_texts = text_splitter.split_documents(documents)

In [14]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splitted_texts, 
                                    embedding=embedding_model,
                                    ids=[str(i) for i in range(len(splitted_texts))]
                                    )

print(vectorstore._collection.count())
jfull_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

182


In [16]:
def do_retrieval(retriever, query):
    result = retriever.get_relevant_documents(query)
    result_docs = ""
    jid = []
    for doc in result:
        jid.append(doc.metadata['jid'])
        result_docs += f"\n案件編號(jid)：{doc.metadata['jid']}\n"
        result_docs += f"{doc.page_content}\n\n"
    jid = list(set(jid))
    jid_string = ""
    for idx, per_jid in enumerate(jid):
        jid_string += f"{idx+1}. {per_jid}\n"
    retrieval_result = "參考資料 - 過往相似案件：" + jid_string + result_docs + "\n" + "="*10
    return retrieval_result

In [17]:
query = "我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷"
print(do_retrieval(jfull_retriever, query))

參考資料 - 過往相似案件：1. TYDV,106,重訴,380,20190517,1
2. CDEV,108,橋簡,116,20190521,1
3. CCEV,107,潮原簡,10,20190506,1

案件編號(jid)：TYDV,106,重訴,380,20190517,1
等規定；原告亦應注意汽車行駛時，駕駛人應注意車前狀況及兩車併行之間隔，並隨時採取必要之安全措施之規定，且依當時為天候晴，日間自然光線，柏油路面乾燥無缺陷，無障礙物，視距良好等，均無不能注意之情事，被告竟於遲至交岔路口時才開啟右方向燈，且未禮讓直行車先行，即貿然搶先右轉，原告亦貿然直行，且未注意車前狀況，致系爭汽車右前車頭與系爭機車之左側車身發生碰撞，系爭機車人車倒地，原告因此受有外傷性顱內出血、臉部挫擦傷、四肢多處挫擦傷、胸部挫傷與左眼玻璃體混濁等傷害；除刑事判決將日期誤繕為105年3月2日，其餘部分原則上為兩造所不爭執，原告並表示刑事判決未將其牙齒傷勢認定於其內，（見本院卷一第171頁），被告上開行為並經本院刑事庭依上開情節依過失傷害罪判處被告有期徒刑3月確定，有本院105年度桃交簡字第1424號判決在卷可參（見本院卷一第5至9頁），並經本院職權調閱卷宗確認無訛，是上情本堪認定。另兩造就本件車禍之過失程度經鑑定認被告駕駛自用小客車行經無號誌交岔路口右轉彎，未依規定距離顯示方向燈光，且未讓同向右側直行車先行，為肇事主因，原告駕駛普通輕機車行經無號誌交岔路口行駛路肩未減速慢行做隨時停車之準備，且未充分注意車前狀


案件編號(jid)：CDEV,108,橋簡,116,20190521,1
、無缺陷、無障礙物、視距良好，並無不能注意之情事，竟疏未注意上情即貿然左轉，適原告騎乘車牌號碼000-000號普通重型機車（下稱系爭車輛），沿後昌路由西往東直行至被告車輛所在之對向車道，見狀閃避不及，兩車因而發生碰撞（下稱系爭事故），致原告受有左手腕關節扭挫傷之傷害（下稱系爭傷害）。原告因系爭事故支出㈠醫療費用新臺幣（下同）150元；㈡系爭車輛修理費用11,300元（即零件費用8,600元、工資2,700元）；㈢薪水損失16,240元；㈣營業損失42,000元；且因精神上痛苦而受有㈤非財產上損害50,000元，合計損失119,690元，被告自應負賠償責任。為此，爰依侵權行為之法律關係，提起本件訴訟等語。並

/opt/conda/envs/verdict/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
def decide_retrieval(query):
    decide_prompt = """你負責判斷"用戶輸入的問題是否與判決、車禍案件有關，並且要去判決資料庫進行搜尋"，
如果需要，則回傳 1
不需要，則回傳 0
以 JSON 格式回傳，key 為 search_required

請注意：只要 0 或 1，不要生成其他文字。

用戶問題：{query}"""
    decide_retrieval_prompt = ChatPromptTemplate.from_template(decide_prompt)
    decide_chain = decide_retrieval_prompt | llm | JsonOutputParser()
    
    return decide_chain.invoke({"query": query})

In [19]:
query = "我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎"
decide_retrieval(query=query)

{'search_required': 1}

In [20]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import HumanMessagePromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

verdict_qa_prompt_message = [
    SystemMessage(content="""你是一個提供判決建議，進行 question-answering tasks 的 AI 助手，你會有一些過往相似的案件，你基於過往類似的案件情況回答現在客戶的問題。
1. 如果客戶問的問題或著你並不知道答案，請直接回答你不知道，
2. 如果客戶問題與過往相似案件無關，則忽略過往相似案件。
3. 使用最多五句話，並且回答簡潔
請注意以上三點。

think step by step
。"""),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("""{context}\n{question}""")
]

verdict_qa_prompt = ChatPromptTemplate.from_messages(verdict_qa_prompt_message)
verdict_rag_chain = (
    {
     "question": itemgetter("question"),
     "context": itemgetter("context"),
     "chat_history": itemgetter("chat_history")}
    | verdict_qa_prompt
    | llm
    | StrOutputParser()
)


chat_history = []
input_query = input('User: ')

while input_query.lower() != 'bye':
    if input_query:
        print(f"user: {input_query}\n")
        if decide_retrieval(query=input_query)['search_required'] == 1:
            print(".... LLM 搜尋歷史相似判決案例中 ....")
            retrivel_result = do_retrieval(jfull_retriever, query)
        else:
            retrivel_result = ""

        # print(f"搜尋結果：{'-'*20}\n{retrivel_result}\n{'-'*20}\n")
        response = ""
        print("AI: ", end = "")
        async for chunk in verdict_rag_chain.astream({"question": input_query,
                                                      "context": retrivel_result,
                                                    "chat_history": chat_history}):
                response += chunk
                print(chunk, end="", flush=True)

        chat_history.append(HumanMessage(content=input_query))
        chat_history.append(AIMessage(content=response))
        print("\n" + '='*20 + "\n")
    input_query = input('User: ')
    
"""
demo question
first round --
  我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。
  我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，
  1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid
  2. 這種情況可以求償什麼項目。
second round --
  我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎
third round --
  法官會依據什麼資訊斟酌精神慰撫金額的賠償依據
"""

user: 我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。   我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，   1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid   2. 這種情況可以求償什麼項目。

.... LLM 搜尋歷史相似判決案例中 ....
AI: 根據您所提供的案件資訊，我為您找到兩個與您情況類似的判決，其案件編號分別為 TYDV,106 和 CCEV,107：
1. 案件編號(TYDV,106，重訴，380,20190517,1):
此案件是關於被告因過失傷害罪而遭起訴。原告主張其因被告的過失行為導致受有傷害，並請求損害賠償。本案判決書中提及，原告支出之交通費用、醫療費用等均屬可請求之項目。
2. 案件編號(CCEV,107，潮原簡，10,20190506,1):
此案件是關於被告因過失傷害罪而遭判處拘役35日，可易科罰金。本案中，原告亦主張其因被告的過失行為導致受有傷害，並請求損害賠償。根據判決書內容，原告可請求之項目包括醫療費用、工作收入損失等。
基於您所提供的情況，我建議您可向對方提出與上述案件類似之索賠項目，例如：
1. 交通費用：包含往返醫院及診所的計程車費或油資費。
2. 醫療費用：包括診斷、治療、藥物及後續復健等費用。
3. 工作收入損失：若因傷無法工作，導致收入減少，可請求此部分之損害賠償。
4. 非財產上損害（精神慰撫金）：依據民法第195條規定，被害人得請求非財產上之損害賠償，即精神慰撫金。法院將依據實際情況，包括傷勢嚴重程度、治療期間、雙方經濟狀況等因素來判定。
請注意，每起案件的細節和條件均不相同，上述建議僅供參考。在進行訴訟前，您可先諮詢律師以獲得更專業且量身打造的建議。

user: 我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎

.... LLM 搜尋歷史相似判決案例中 ....
AI: 當然，我很樂意協助您。根據您所提供的判決摘要，我為各位整理出重點如下：
1. TYDV,106 重訴 380 20190517 1:
	* 原告因被告過失傷害行為導致受傷，請求損害賠償。
	* 可求償項目包括交通費用、醫療費用、工作收入損失及非財產上損害（精神慰撫金）。

'\ndemo question\nfirst round --\n  我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。\n  我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，\n  1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid\n  2. 這種情況可以求償什麼項目。\nsecond round --\n  我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎\nthird round --\n  法官會依據什麼資訊斟酌精神慰撫金額的賠償依據\n'

In [22]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import HumanMessagePromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

verdict_qa_prompt_message = [
    SystemMessage(content="""你是一個提供判決建議，進行 question-answering tasks 的 AI 助手，你會有一些過往相似的案件，你基於過往類似的案件情況回答現在客戶的問題。
1. 如果客戶問的問題或著你並不知道答案，請直接回答你不知道，
2. 如果客戶問題與過往相似案件無關，則忽略過往相似案件。
3. 使用最多五句話，並且回答簡潔
請注意以上三點。

think step by step
。"""),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("""{context}\n{question}""")
]

verdict_qa_prompt = ChatPromptTemplate.from_messages(verdict_qa_prompt_message)
verdict_rag_chain = (
    {
     "question": itemgetter("question"),
     "context": itemgetter("context"),
     "chat_history": itemgetter("chat_history")}
    | verdict_qa_prompt
    | llm
    | StrOutputParser()
)


chat_history = []
input_query = input('User: ')

while input_query.lower() != 'bye':
    if input_query:
        print(f"user: {input_query}\n")
        # if decide_retrieval(query=input_query)['search_required'] == 1:
        #     print(".... LLM 搜尋歷史相似判決案例中 ....")
        #     retrivel_result = do_retrieval(jfull_retriever, query)
        # else:
        #     retrivel_result = ""

        retrivel_result = ""
        response = ""
        print("AI: ", end = "")
        async for chunk in verdict_rag_chain.astream({"question": input_query,
                                                      "context": retrivel_result,
                                                    "chat_history": chat_history}):
                response += chunk
                print(chunk, end="", flush=True)

        chat_history.append(HumanMessage(content=f"{retrivel_result}\n{input_query}"))
        chat_history.append(AIMessage(content=response))
        print("\n" + '='*20 + "\n")
    input_query = input('User: ')
    
"""
demo question
first round --
  我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。
  我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，
  1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid
  2. 這種情況可以求償什麼項目。
second round --
  我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎
third round --
  法官會依據什麼資訊斟酌精神慰撫金額的賠償依據
"""

user: 我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。   我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，   1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid   2. 這種情況可以求償什麼項目。

AI: 1.基於所提的情境，與你狀況相近的判例可能包括：(a)州法關於車輛未依規定讓直行車先行、(b)車輛在交叉路口左轉時未遵守交通號誌或標誌指示、(c)因疏忽或不當駕駛導致的碰撞等。
2.你可能可以求償的項目，取決於你的保險政策及對方肇事責任程度，項目可能包括：(a)醫療費用及復健;(b)收入損失；(c)疼痛、不適、及其他因傷所致的非財務損害(這些通常是以非財務損害或痛苦索賠計算);(d)車輛修理或替換。
請注意，我提供的資訊是概括性的，無法取代專業法律或保險顧問的建議。

user: 我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎

AI: 當然，我非常樂意協助您。由於我無法存取網路，也無法存取特定司法管轄區的判例，因此我無法提供具體的判例給您參考。不過我可以提供一般性案件概要，說明類似情況可能涉及的法律議題。

1. 《蘇利文 v.紐約州》(Sullivan v. New York)，這是一起發生在紐約州的案件，原告聲稱被告未依規定讓直行車先行而撞上他。法院最終判決，被告必須負責，因為他未遵守交通法規。
2. 《菲利浦斯 v.菲利浦斯》(Phillips v. Phillips)，這是一起英國的案件，夫妻雙方在交叉路口發生碰撞，導致一方受傷。法院判決肇事者須負責，因為他未遵守交通號誌。
3. 《巴傑納 v.雷耶斯》(Bajada v. Reyes)，這是一起菲律賓的案件，原告聲稱被告在交叉路口左轉時未遵守交通號誌而撞上他。法院判決，被告須負責，因為他未遵守交通法規。

這些案例只是概要說明，在類似情況下可能涉及的法律議題。在評估您的案件時，重要的是考慮具體的事實和證據，以及所適用的交通法規。由於我無法存取網路，也無法存取特定司法管轄區的判例，因此建議您諮詢專精交通法的律師，以獲得有關您的案件具體法律議題的具體建議。

user: 法官會依據什麼資訊斟酌精神慰撫金額的賠償依據

AI: 判定非財務損

'\ndemo question\nfirst round --\n  我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。\n  我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，\n  1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid\n  2. 這種情況可以求償什麼項目。\nsecond round --\n  我應該有提供給你一些可以參考的判決，可以列給我看並且幫我摘要這些案件的案發經過嗎\nthird round --\n  法官會依據什麼資訊斟酌精神慰撫金額的賠償依據\n'

### **TopK Result(Retrivel 結果排名)**

**RAG-Fusion**

1. User 給 Query
2. LLM 猜測 User 可能想問的問題，並拆解成多個問題
3. 分別進行資料庫檢索
4. 針對所有的查詢結果做相對排名




In [24]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [27]:
from langchain.load import dumps, loads
from typing import List

def reciprocal_rank_fusion(results: List[List], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results



query = """我騎著我的機車往北方行駛，突然一輛車經過。那輛車本應該停下來讓直行的車輛先過，但它沒有停，就這樣直接左轉進了交岔路口。
  我來不及反應，只好試圖閃避，但最終還是和它撞上了。這場事故讓我右肩脫臼，身上也有很多挫傷，
  1. 跟我情況類似的判決有什麼，請提供我這些相似判決的 jid
  2. 這種情況可以求償什麼項目"""
  
  
generated_query = generate_queries.invoke(query)
  
retrieval_chain_rag_fusion = generate_queries | jfull_retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": query})

In [28]:
print(generated_query)

['1. 我騎著機車遇到交通事故的判決案例', '2. 汽車未讓直行車輛先過而導致事故的法律責任', '3. 求償交通事故中的身體傷害賠償項目', '4. 機車與汽車交通事故的法律程序和賠償方式']


In [29]:
docs

[(Document(page_content='行經上開交岔路口時左轉，肇事車輛右前車頭因而與系爭車輛右後車尾發生碰撞，致原告受有前胸挫傷、頸部扭傷等傷害。為此原告爰依侵權行為之法律關係請求被告賠償㈠車輛修復費用新臺幣（下同）5,700元（零件費用）；㈡精神慰撫金300,000元；㈢因傷沒有工作3個月，以一個月40,000元計算，工作損失共120,000元（計算式：40,000元ㄨ3個月＝120,000元）；㈣醫療費74,300元，綜上合計500,000元（計算式：5,700元＋300,000元＋120,000元＋74,300元＝500,000元）。並聲明：㈠被告應給付原告500,000元，及自起訴狀繕本送達翌日起（即107年10月24日起）至清償日止，按年息5％計算之利息。㈡原告願供擔保，請准宣告假執行。二、被告則以：對於車輛修理費5,700元、一個月工資40,000元且三個月沒有工作、醫療費74,300元等均否認。另就醫療費收據部分，其中收據20元那張距離本件事故發生已經一年，而另外一張門診收據1,670元那張沒有意見等語置辯。聲明：原告之訴駁回。三、按「特種閃光號誌各燈號顯示之意義如左：一、閃光黃燈表示『警告』，車輛應減速接近，注意安全，小心通過。．', metadata={'jid': 'CCEV,108,潮簡,114,20190509,1'}),
  0.03279569892473118),
 (Document(page_content='、無缺陷、無障礙物、視距良好，並無不能注意之情事，竟疏未注意上情即貿然左轉，適原告騎乘車牌號碼000-000號普通重型機車（下稱系爭車輛），沿後昌路由西往東直行至被告車輛所在之對向車道，見狀閃避不及，兩車因而發生碰撞（下稱系爭事故），致原告受有左手腕關節扭挫傷之傷害（下稱系爭傷害）。原告因系爭事故支出㈠醫療費用新臺幣（下同）150元；㈡系爭車輛修理費用11,300元（即零件費用8,600元、工資2,700元）；㈢薪水損失16,240元；㈣營業損失42,000元；且因精神上痛苦而受有㈤非財產上損害50,000元，合計損失119,690元，被告自應負賠償責任。為此，爰依侵權行為之法律關係，提起本件訴訟等語。並聲明：被告應給付原告119,690元，及自刑事附帶民事起訴狀繕本送達之翌日起至清償日止，按週年利率5%計算之利息